<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

In [6]:
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt

# <a id='climate_graph'></a>Example - Interactive climate graph application

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import ee

import plotly.graph_objs as go


In [ ]:
ee.Initialize()

In [ ]:
era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY')

In [ ]:
era5_monthly_img = era5_monthly.limit(1).first()
collection_img_proj = era5_monthly_img.select(0).projection()
#era5_monthly_img.getInfo()

In [ ]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = era5_monthly.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
#img_list[0].getInfo()

In [ ]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
#meanMonths_collection.getInfo()

In [ ]:
longitude =12.5
latitude = 41.9

In [ ]:
point = ee.Geometry.Point(longitude,latitude)
tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()


In [ ]:
ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    
ydata_t2m = [row[4]-273.15 for row in t2m_point[1:]]

In [ ]:
def visualize_climate_graph():    
    point = ee.Geometry.Point(lon.value, lat.value)
    tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()

    ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    ydata_t2m = [row[4]-273.2 for row in t2m_point[1:]]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat.value,2)) + ' / '+ str(round(lon.value,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm",
            dtick=20
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[min(ydata_t2m)-2,max(ydata_t2m)+2],
            dtick=5
        ),
        plot_bgcolor='white',
        legend=dict(
            orientation='h')
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
lat = widgets.FloatSlider(
    value=41.9,
    min=-90.0,
    max=90.0,
    step=0.25,
    description='Latitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal', #horizontal
    readout=True
)


In [ ]:
lon = widgets.FloatSlider(
    value=12.5,
    min=-180.0,
    max=180.0,
    step=0.25,
    description='Longitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)


In [ ]:
def click(b):
    fig = visualize_climate_graph()
    with out:
        clear_output(wait=True)
        fig.show()

In [ ]:
button=widgets.Button(description='Plot climate graph')
button.on_click(click)

In [ ]:
out=widgets.Output()
display(widgets.HBox([lat,lon]))
display(button)
display(out)

<br>

<hr>
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>